## Décorateurs, Générateurs, Iterateurs, Async/Await en Python et plus encore !

## Décorateurs (@)

> Analogie : C'est comme amballer un cadeau (votre fonction originale) reste le meme, mais vous lui ajoutez du papier cadeau et un ruban (la nouvelle fonctionnalité)

In [ ]:
# Imaginons nous voulions mesurer le temps d"exécution d'une fonction.
import time

def timer_decorateur(func):
    # La fonction "wrapper" enveloppe la fonction originale
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs) # on exécute la fonction originale
        end_time = time.time()
        print(f"{func.__name__} a pris {end_time - start_time} secondes pour s'exécuter.")
        return result
    return wrapper

@timer_decorateur
def longue_operation(nom_tache):
    time.sleep(2)
    print(f"Tache '{nom_tache} terminée !")
    
# on appelle simplement la fonction, le décorateur est déjà appliqué !
longue_operation("Nettoyage de données")
    

Tache 'Nettoyage de données terminée !
longue_operation a pris 2.001107692718506 secondes pour s'exécuter.


In [1]:
# Décorateur de journalisation (logging) : ce décorateur affiche des informations sur une focntion (ses arguments et sa valeur de retour) chaque fois qu'elle est appelée.
def log_function_call(func):
    # Affiche le nom de fonction, ses arguments et sa valeur de retour.
    def wrapper(*args, **kwargs):
        # avant l'appel de la fonction
        print(f"Appel de la fonction '{func.__name__}'...")
        print(f"Arguments positionnels (args): {args}")
        print(f"Arguments nommés (kwargs): {kwargs}")
        
        # Appel de la fonction originale
        result = func(*args, **kwargs)
        
        # Aprés l'appel de la fonction
        print(f"La fonction '{func.__name__}' a retourné : {result}")
        return result
    return wrapper

@log_function_call
def addition(a, b):
    return a+b

@log_function_call
def saluer(nom, message="Bonjour"):
    return f"{message}, {nom} !"

# utilisation
addition(3,5)
saluer("Simone")

Appel de la fonction 'addition'...
Arguments positionnels (args): (3, 5)
Arguments nommés (kwargs): {}
La fonction 'addition' a retourné : 8
Appel de la fonction 'saluer'...
Arguments positionnels (args): ('Simone',)
Arguments nommés (kwargs): {}
La fonction 'saluer' a retourné : Bonjour, Simone !


'Bonjour, Simone !'

## Les Générateurs (yield)


In [ ]:
# Fonction classique (gourmande en mémoire pour de grandes listes)
def liste_carres(n):
    resultats = []
    for i in range(n):
        resultats.append(i * i)
    return resultats

# Générateur (très efficace en mémoire)
def generateur_carres(n):
    for i in range(n):
        yield i * i # 'yield' met en pause et "retourne une valeur"
        
# use case
print("Générateur")
# 'gen' est un objet générateur, rien n'a encore été calculé
gen = generateur_carres(10)
print(gen)

# on consommme les valeurs une par une
print(next(gen)) # Affiche 0
print(next(gen)) # Affiche 1
print(next(gen)) # Affiche 4

# on peut aussi l'utiliser directement dans une boucle for
print("Reste des valeurs :")
for carre in gen:
    print(carre) # 9 16 etc....

Générateur
<generator object generateur_carres at 0x00000285DB908110>
0
1
4
Reste des valeurs :
9
16
25
36
49
64
81


> Un générateur est une fonction qui ne retourne pas une valeur unique, mais une séquence de valeurs à la demande, en utilisant le mot-clé yield.
Contrairement à une fonction classique qui retourne un résultat et termine, un générateur se met en pause après chaque yield et reprends là où il s’était arrêté quand on lui demande la valeur suivante.

## Comparaison entre une liste et un générateur

In [ ]:
# Création d'une liste avec 1 million de nombres
grande_liste = liste_carres(1000000) # consomme beaucoup de mémoire (RAM)  IMPORTANT : NE PAS EXECUTER CETTE CELLULE SI VOUS AVEZ PEU DE RAM

Problème :

- La liste occupe beaucoup de mémoire (environ 8 Mo pour 1 million d’entiers).
- Si tu ne traites qu’un élément à la fois, c’est du gaspillage.

In [4]:
# Avec un générateur, on peut traiter les nombres un par un sans tout stocker en mémoire
def generate_nombres(n):
    for i in range(n):
        yield i # 'yield' met en pause et "retourne une valeur"
        
nombre = generate_nombres(1000000) # ici pas de liste en mémoire, juste un générateur créé
for _ in range(10): # on traite les 10 premiers nombres
    print(next(nombre)) # affiche les nombres de 0 à 9

0
1
2
3
4
5
6
7
8
9


Avantages :

- Pas de stockage en mémoire : Les nombres sont générés à la volée.
- Économie de mémoire : Seul le nombre courant est en mémoire.
- Traitement immédiat : Pas besoin d’attendre que tout soit généré.

In [ ]:
#  Traitement "Lazy" (paresseux)
# Les données sont générées uniquement quand on en a besoin.
def lire_gros_fichier(fichier):
    with open(fichier, 'r') as f:
        for ligne in f:  # ⬅️ Générateur implicite !
            yield ligne

for ligne in lire_gros_fichier("huge_file.txt"):
    print(ligne)  # Traite une ligne à la fois

### Exemple Avancé : Pipeline de données avec des génréteurs

> Alors, imaginons que nous ayons une flux de données brutes (ventes, logs, etc...) et nous voulons les nettoyer, les filtrer et les transformer, le tout sans jamais rien stocker en mémoire.

In [5]:
# Nos données sources simulées

donnees_brutes = [
    {"id": 1, "produit": "Pomme", "quantite": 10},
    {"id": 2, "produit": "Orange", "quantite": "cinq"}, # Donnée invalide
    {"id": 3, "produit": "Banane", "quantite": 150},
    {"id": 4, "produit": "Poire", "quantite": 8},
    {"id": 5, "produit": "Fraise", "quantite": 200},
    {"id": 6, "produit": "Raisin", "quantite": 50},
    {"id": 7, "produit": "Melon", "quantite": 12},
    {"id": 8, "produit": "Abricot", "quantite": -5},            # quantite négative (invalide)
    {"id": 9, "produit": "Kiwi", "quantite": "douze"},         # quantite en chaîne (invalide)
    {"id": 10, "produit": "Mangue", "quantite": 30},
    {"id": 11, "produit": "Cerise", "quantite": None},         # valeur manquante (invalide)
    {"id": 12, "produit": "Ananas"},                           # clé quantite manquante (invalide)
    {"id": 13, "produit": "  Poire  ", "quantite": "15"},      # espaces et quantite en chaîne (à normaliser)
]

# Étape 1 : Nettoyage des données, premier générateur
def nettoyer_donnees(donnees):
    # générateur qui nettoie les données
    print("Début du nettoyage des données...")
    for item in donnees:
        try:
            quantite = int(item.get("quantite", 0)) # Convertir en entier, défaut à 0 si manquant
            if quantite < 0:
                continue  # Ignorer les quantités négatives
            item["quantite"] = quantite # Mettre à jour la quantité nettoyée
            item["produit"] = item["produit"].strip().title()  # Normaliser le nom du produit
            yield item # ne produit l'item si il est valide
        except (ValueError, TypeError):
            print(f"Donnée invalide ignorée : {item}")
            continue  # Ignorer les entrées avec des quantités non convertibles
        
# Étape 2 : Filtrage des données, deuxième générateur
def filtrer_donnees(donnees, seuil_quantite):
    print(f"Filtrage des données avec un seuil de quantité > {seuil_quantite}...")
    for item in donnees:
        if item["quantite"] > seuil_quantite:
            yield item
        else:
            print(f"Donnée filtrée (quantité insuffisante) : {item['produit']} avec quantité {item['quantite']}")

# Étape 3 : Transformation des données, troisième générateur
def transformer_donnees(donnees):
    print("Transformation des données...")
    for item in donnees:
        item_transforme = {
            "ID Produit": item["id"],
            "Nom Produit": item["produit"],
            "Quantité en Stock": item["quantite"],
            "Statut": "En Stock" if item["quantite"] > 0 else "Rupture de Stock"
        }
        yield item_transforme
    
# Pipeline complet
pipeline = transformer_donnees(
    filtrer_donnees(
        nettoyer_donnees(donnees_brutes),
        seuil_quantite=10
    )
)

# Consommation du pipeline
for produit in pipeline:
    print(produit)
    

Transformation des données...
Filtrage des données avec un seuil de quantité > 10...
Début du nettoyage des données...
Donnée filtrée (quantité insuffisante) : Pomme avec quantité 10
Donnée invalide ignorée : {'id': 2, 'produit': 'Orange', 'quantite': 'cinq'}
{'ID Produit': 3, 'Nom Produit': 'Banane', 'Quantité en Stock': 150, 'Statut': 'En Stock'}
Donnée filtrée (quantité insuffisante) : Poire avec quantité 8
{'ID Produit': 5, 'Nom Produit': 'Fraise', 'Quantité en Stock': 200, 'Statut': 'En Stock'}
{'ID Produit': 6, 'Nom Produit': 'Raisin', 'Quantité en Stock': 50, 'Statut': 'En Stock'}
{'ID Produit': 7, 'Nom Produit': 'Melon', 'Quantité en Stock': 12, 'Statut': 'En Stock'}
Donnée invalide ignorée : {'id': 9, 'produit': 'Kiwi', 'quantite': 'douze'}
{'ID Produit': 10, 'Nom Produit': 'Mangue', 'Quantité en Stock': 30, 'Statut': 'En Stock'}
Donnée invalide ignorée : {'id': 11, 'produit': 'Cerise', 'quantite': None}
Donnée filtrée (quantité insuffisante) : Ananas avec quantité 0
{'ID Prod